## NN Structure

output policy will have a dimension of the sum of discard choices, play choices, and clue choices. clue choices has a cardinality of product of num players, num colors, and num numbers. In this case it will be 4+4+3*3*1= 17 element array

Input layer will have a dimension of the sum of opp_hand, board, discard, self_clue, and opp_clue.
opp_hand will be dummy encoding of the number of cards in the deck. board is the number of playable cards. self_clue is product of num of cards_in_hand, num of colors, and num of value, so 4*3*3 = 36. Same with opp_clue

Input layer = 15+9+15+36+36 = 111 node
also add in num bombs, number of cards left, number of clues, 4, 15-4*2=7, 8

total input layer = 111+4+7+8 = 130


In [1]:
import random
import numpy as np
from copy import deepcopy
from Game import *

In [2]:
from keras.utils import plot_model
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(183,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(90, activation='relu')(inputs)
x = Dense(45, activation='relu')(x)
predictions = Dense(14, activation='softmax')(x)
value = Dense(1, activation='softmax')(x)
# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=[predictions,value])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# model.fit(data, labels)  # starts training
plot_model(model, to_file='model.png',show_shapes = True)

C:\Users\yidin\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [31]:
def game_act(agent, opp, deck, board, num):
    done = 0
#     try:
    if num < 4:
        discard = agent.play_card(num)
        board.discard(discard)
        agent.draw(deck.deal(1))
        board.add_clue()
        print("Action ",agent.name," discarded: %s"%(discard))
    elif num < 8:
        play= agent.play_card(num-4)
        agent.draw(deck.deal(1))
        board.play_card(play)
        print("Action ",agent.name," played: %s"%(play))
    elif num< 14:
        c = [1,2,3,"R","B","Y"]
        info = c[num-8]
        opp.receive_clue(info)
        board.remove_clue()
        print("Action ",agent.name," gave clue: %s to %s"%(str(info),opp.name))
#         board.turns +=1
#     except:
#         done = 1

    if deck.count == 0:
        done = 1
        print(done)

    return(done, deck, agent, [opp] ,board)

In [27]:
class GS():
    def __init__(self, board, deck, curr, opps, turn):
        self.board = board
        self.curr = curr
        self.opps = opps
        self.turn = turn
        self.deck = deck
        self.id = str(curr.nn_input(opps[0], board))
        print(self.board.score)
        
    def takeAction(self, action):
        done, deck, curr, opps, board = game_act(self.curr.copy(), self.opps[0].copy(), 
                                                  self.deck.copy(), self.board.copy(), action)
        print("Deck", [str(c) for c in deck.cards])
        newState = GS(board, deck, opps[0], [curr], self.turn+1)
        return(newState, board.score, done)
    
    def _print(self):
        print("")

class Node():
    def __init__(self, state):
        self.state = state
        self.id = state.id
        self.edges = []
    
    def isLeaf(self):
        if len(self.edges) > 0:
            return False
        else:
            return True        
class Edge():

    def __init__(self, inNode, outNode, prior, action):
        self.id = inNode.state.id + '|' + outNode.state.id
        self.inNode = inNode
        self.outNode = outNode
        self.action = action
        
        self.stats =  {
            'N': 0,
            'W': 0,
            'Q': 0,
            'P': prior,}
        
class MCTS():
    def __init__(self, root, cpuct):
        self.root = root
        self.tree = {}
        self.cpuct = cpuct
        self.addNode(root)
        
    def __len__(self):
        return len(self.tree)


    def moveToLeaf(self):
        breadcrumbs = []
        currentNode = self.root

        done = 0
        value = 0
        
        while (not currentNode.isLeaf()) and (not done ==1):
            maxQU = -99999
            if currentNode == self.root:
                epsilon = 0.2
                nu = np.random.dirichlet([0.8] * len(currentNode.edges))
            else:
                epsilon = 0
                nu = [0] * len(currentNode.edges)
                
            Nb = 0
            for action, edge in currentNode.edges:
                Nb = Nb + edge.stats['N']

            for idx, (action, edge) in enumerate(currentNode.edges):
                U = self.cpuct * \
                    ((1-epsilon) * edge.stats['P'] + epsilon * nu[idx] )  * \
                    np.sqrt(Nb) / (1 + edge.stats['N'])
                
                Q = edge.stats['Q']

                if Q + U > maxQU:
                    maxQU = Q + U
                    simulationAction = action
                    simulationEdge = edge

    
            newState, value, done = currentNode.state.takeAction(simulationAction) #the value of the newState from the POV of the new playerTurn
            currentNode = simulationEdge.outNode
            breadcrumbs.append(simulationEdge)
            print(newState.id,"TURN  ",newState.turn)
        return currentNode, value, done, breadcrumbs



    def backFill(self, leaf, value, breadcrumbs):
        for edge in breadcrumbs:
            edge.stats['N'] = edge.stats['N'] + 1
            edge.stats['W'] = edge.stats['W'] + value
            edge.stats['Q'] = edge.stats['W'] / edge.stats['N']
            
    def addNode(self, node):
        self.tree[node.id] = node

In [28]:
print("ASdf")
class Player():
    def __init__(self, name, deck, model):
        self.deck = deck
        self.name = name
        self.hand = []
        self.hand_binary = [0] * len(self.deck.cards)
        self.private_clues = [clue(self.deck.cards), clue(self.deck.cards), clue(self.deck.cards), clue(self.deck.cards)]
        self.public_clues = [clue(self.deck.cards), clue(self.deck.cards), clue(self.deck.cards), clue(self.deck.cards)]
        self.seen = []
        self.seenplayer_binary = [0] * len(self.deck.cards)
        self.played = []
        self.seenboard_binary = [0] * len(self.deck.cards) 
        
        self.cpuct = 0.8
        self.MCTSsimulations = 50
        self.model = model
        self.mcts = None
        
        self.train_overall_loss = []
        self.train_value_loss = []
        self.train_policy_loss = []
        self.val_overall_loss = []
        self.val_value_loss = []
        self.val_policy_loss = []
    
    def copy(self):
        c = Player(self.name, self.deck.copy(), self.model)
        c.hand = self.hand.copy()
        c.hand_binary = self.hand_binary.copy()
        c.private_clues = self.private_clues.copy()
        c.public_clues = self.public_clues.copy()
        c.seen = self.seen.copy()
        c.seenplayer_binary = self.seenplayer_binary.copy()
        c.played = self.played.copy()
        c.seenboard_binary = self.seenboard_binary.copy()

        c.model = self.model
        c.mtcs = self.mcts

        return(c)
    
    def print_hand(self):
        print(self.name, [str(c.value) + c.color for c in self.hand])

    def draw(self, cards):
        self.hand += cards
        self.hand_binary = to_bin(self.hand, self.deck.cards)

    def play_card(self, pos):
        played = self.hand[pos]
        del self.hand[pos]
        return played

    def discard_card(self, pos):
        played = self.hand[pos]
        del self.hand[pos]
        return played

    def see_players(self, players=[]):
        for p in players:
            self.seen += p.hand

        self.seen = list(set(self.seen))
        self.seenplayer_binary = to_bin(self.seen, self.deck.cards)
        print("%s has seen:" % (self.name), [x.__str__() for x in self.seen])

        for c in self.private_clues:
            add_clue = to_bin(self.seen, self.deck.cards)
            add_clue = [-x for x in add_clue]
            c.clues = [min(c.clues[j], add_clue[j]) for j in range(0, len(c.clues))]

    def see_board(self, board):
        self.played += board.playable
        self.played = list(set(self.played))
        self.seenboard_binary = to_bin(self.played, self.deck.cards)
        print("%s has seen played:" % (self.name), [x.__str__() for x in self.seen])

        for i in range(0, 4):
            self.public_clues[i].clues = [-1 if x in self.played else 0 for x in self.deck.cards]
            self.private_clues[i].clues = [-1 if x in self.played else 0 for x in self.deck.cards]

    def receive_clue(self, clue):
        i = 0

        if type(clue) == str:
            valid = [c for c in self.hand if c.color == clue]
            if not valid == []:
                for card in self.hand:
                    c = self.public_clues[i]
                    if card.color != clue:
                        add_clue = [-1 if x.color == clue else 0 for x in self.deck.cards]
                    else:
                        add_clue = [-1 if x.color != clue else 0 for x in self.deck.cards]

                    c.clues = [min(c.clues[j], add_clue[j]) for j in range(0, len(c.clues))]
                    priv = self.private_clues[i]
                    priv.clues = [min(c.clues[j], priv.clues[j]) for j in range(0, len(c.clues))]
                    i += 1

        if type(clue) == int:
            valid = [c for c in self.hand if c.value == clue]
            if not valid == []:
                for card in self.hand:
                    c = self.public_clues[i]
                    if card.value != clue:
                        add_clue = [-1 if x.value == clue else 0 for x in self.deck.cards]
                    else:
                        add_clue = [-1 if x.value != clue else 0 for x in self.deck.cards]

                    c.clues = [min(c.clues[j], add_clue[j]) for j in range(0, len(c.clues))]
                    priv = self.private_clues[i]
                    priv.clues = [min(c.clues[j], priv.clues[j]) for j in range(0, len(c.clues))]
                    i += 1

    def nn_input(self, opp, board):
        self_clues = []
        for c in self.private_clues:
            self_clues += c.clues

        opp_clues = []
        for c in opp.public_clues:
            opp_clues += c.clues

        inputs = self.seenplayer_binary + \
                 self.seenboard_binary + \
                 board.discard_bin + \
                 board.playable_bin + \
                self_clues + \
                 opp_clues + \
                 [board.bombs, board.clues, board.score]
                    
        return (inputs)
    
    def simulate(self):
        ##### MOVE THE LEAF NODE
        leaf, value, done, breadcrumbs = self.mcts.moveToLeaf()
        
        ##### EVALUATE THE LEAF NODE
        value, breadcrumbs = self.evaluateLeaf(leaf, value, done, breadcrumbs)
        
        ##### BACKFILL THE VALUE THROUGH THE TREE
        self.mcts.backFill(leaf, value, breadcrumbs)


    def act(self, state, tau):
        if self.mcts == None or state.id not in self.mcts.tree:
            self.buildMCTS(state)
        else:
            self.changeRootMCTS(state)

        #### run the simulation
        for sim in range(self.MCTSsimulations):
            self.simulate()

        #### get action values
        pi, values = self.getAV(1)

        ####pick the action
        action, value = self.chooseAction(pi, values, tau)
        
        nextState, _, _ = state.takeAction(action)
        
        NN_value = self.get_preds(nextState)[0]
        
        return (action, pi, value, NN_value)


    def get_preds(self, state):
        #predict the leaf
        curr = state.curr
        opps = state.opps
        board = state.board
        
        nn_input = curr.nn_input(opps[0],board)
        probs, value = self.model.predict(np.array([nn_input]))
        
        
        return ((probs, value))


    def evaluateLeaf(self, leaf, value, done, breadcrumbs):
        if done == 0:
            probs, value = self.get_preds(leaf.state)
            for action in range(0,14):
                newState, _, _ = leaf.state.takeAction(action)
                if newState.id not in self.mcts.tree:
                    node = Node(newState)
                    self.mcts.addNode(node)
                else:
                    node = self.mcts.tree[newState.id]

                newEdge = Edge(leaf, node, probs[0][action], action)
                leaf.edges.append((action, newEdge))

        return ((value, breadcrumbs))


    def getAV(self, tau):
        edges = self.mcts.root.edges
        pi = np.zeros(14, dtype=np.integer)
        values = np.zeros(14, dtype=np.float32)
        
        for action, edge in edges:
            print(edge.stats)
            pi[action] = pow(edge.stats['N'], 1/tau)
            values[action] = edge.stats['Q']

        pi = pi / (np.sum(pi) * 1.0)
        return pi, values

    def chooseAction(self, pi, values, tau):
        if tau == 0:
            actions = np.argwhere(pi == max(pi))
            action = random.choice(actions)[0]
        else:
            action_idx = np.random.multinomial(1, pi)
            action = np.where(action_idx==1)[0][0]
            
        value = values[action]

        return action, value


    def buildMCTS(self, state):
        self.root = Node(state)
        self.mcts = MCTS(self.root, self.cpuct)

    def changeRootMCTS(self, state):
        self.mcts.root = self.mcts.tree[state.id]

ASdf


In [32]:
cards = [Card(value, color) for color in ["R","B","Y"] for value in [1,1,2,2,3]]
deck = Deck(cards = cards)
p1 = Player("P1", deck.copy(), model)
p2 = Player("P2", deck.copy(), model)
board = Board(["R","B","Y"], deck.copy())
deck.shuffle()
p1.draw(deck.deal(4))
p2.draw(deck.deal(4))



players = [p1, p2]
turn = 0
curr = players[(turn)%2]
opp = players[(turn+1)%2]
curr.see_players([opp])
curr.see_board(board)
        

currGS = GS(board, deck, curr, [opp], turn)

P1 has seen: ['2B', '2Y', '1B', '3R']
P1 has seen played: ['2B', '2Y', '1B', '3R']
0


In [36]:
p1.buildMCTS(currGS)
# for i in range(0,10):
    p1.simulate()

IndentationError: unexpected indent (<ipython-input-36-a7c484a55364>, line 3)

In [26]:
M  = p1.mcts
len(M.tree.keys())

47

In [ ]:
board.discard_bin

In [ ]:
def convert_nn_input(nn_input, cards):
    l = len(cards)
    curr_seenplayer = nn_input[0:l]
    curr_seenboard = nn_input[l:(2*l)]
    board_discard = nn_input[(2*l):(3*l)]
    board_playabale = nn_input[(3*l):(4*l)]
    self_clues = nn_input[(4*l):(8*l)]
    opp_clues = nn_input[(8*l):(12*l)]
    board_bombs = nn_input[12*l+1]
    board_clues = nn_input[12*l+2]
    board_score = nn_input[12*l+3]
    
    print([cards[i] for i in board_playable])
    
    
    
    
        
    
    

In [ ]:
list(M.tree.keys())[12]

In [ ]:
convert

In [ ]:
cards = [Card(value, color) for color in ["R","B","Y"] for value in [1,1,2,2,3]]
p1 = Player("P1", cards, model)
p2 = Player("P2", cards, model)
board = Board(["R","B","Y"], cards)


deck = Deck(cards = cards)
deck.shuffle()
p1.draw(deck.deal(4))
p2.draw(deck.deal(4))

p3 = p1.copy()
p1.print_hand()
done = False
players = [p1, p2]
turn = 0
while not done:
    curr = players[(turn)%2]
    opp = players[(turn+1)%2]
    curr.see_players([opp])
    curr.see_board(board)

    nn_input = curr.nn_input(opp,board)
    probs, _ = model.predict(np.array([nn_input]))
    choice = np.random.choice(range(0,14), p=probs[0])
    done = game_act(curr, opp, deck, board, choice)
    print(done)
    turn +=1
print(board.score)

In [ ]:
p1.print_hand()
p3.print_hand()


In [ ]:
p1.play_card(1)

In [37]:
class X():
    
    def __init__(self, x , y):
        self.x = x
        self.y = y
    
    def update(self, num):
        self.x = num
    def copy(self):
        c = X(self.x, self.y)
        return(c)
    
class Y():
    def __init__(self, Z):
        self.Z = Z


In [38]:
a =X(1,2)
b = Y(a)

In [40]:
b.Z.update(10)

In [43]:
a.x

10